In [ ]:
import jtrace
import os
import numpy as np
from ipywidgets import interact
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
LSST_r_fn = os.path.join(jtrace.datadir, "LSST", "LSST_r.yaml")
telescope = jtrace.Telescope.makeFromYAML(LSST_r_fn)

In [ ]:
for name, surface in telescope.surfaces.items():
    print(name, surface['surface'])

In [ ]:
plt.figure(figsize=(12, 11))

for surface in telescope.surfaces.values():
    x = np.linspace(-surface['outer'], -surface['inner'], 100, endpoint=True)
    plt.plot(x, [surface['surface'].sag(x_,0) for x_ in x], c='k')
    x = np.linspace(surface['inner'], surface['outer'], 100, endpoint=True)
    plt.plot(x, [surface['surface'].sag(x_,0) for x_ in x], c='k')

p0_left = jtrace.Vec3(-4.1, 0, 0)
p0_right = jtrace.Vec3(4.1, 0, 0)
p0_lmid = jtrace.Vec3(-2.9, 0, 0)
p0_rmid = jtrace.Vec3(2.9, 0, 0)
v_left = jtrace.Vec3(-0.025, 0, -1).UnitVec3()/1.000277
v_mid = jtrace.Vec3(0, 0, -1).UnitVec3()/1.000277
v_right = jtrace.Vec3(0.025, 0, -1).UnitVec3()/1.000277

for v, c in zip([v_left, v_mid, v_right], ['#377eb8', '#ff7f00', '#4daf4a']):
    rays = jtrace.RayVector()
    for p0 in [p0_left, p0_lmid, p0_rmid, p0_right]:
        ray = jtrace.Ray(p0, v, t=0, w=500e-9)
        p0 = ray.positionAtTime(-8)        
        rays.append(jtrace.Ray(p0, v, t=0, w=500e-9))
    trace = telescope.traceFull(rays)
    for surface in trace:
        for inray, outray in zip(surface['inrays'], surface['outrays']):
            plt.plot([inray.x0, outray.x0], [inray.z0, outray.z0], c=c)

In [ ]:
plt.figure(figsize=(12, 11))
plt.ylim(2.5, 4.5)
plt.xlim(-0.5, 0.5)

for surface in telescope.surfaces.values():
    x = np.linspace(-surface['outer'], -surface['inner'], 100, endpoint=True)
    plt.plot(x, [surface['surface'].sag(x_,0) for x_ in x], c='k')
    x = np.linspace(surface['inner'], surface['outer'], 100, endpoint=True)
    plt.plot(x, [surface['surface'].sag(x_,0) for x_ in x], c='k')

p0_left = jtrace.Vec3(-4.1, 0, 0)
p0_right = jtrace.Vec3(4.1, 0, 0)
p0_lmid = jtrace.Vec3(-2.9, 0, 0)
p0_rmid = jtrace.Vec3(2.9, 0, 0)
v_left = jtrace.Vec3(-0.025, 0, -1).UnitVec3()/1.000277
v_mid = jtrace.Vec3(0, 0, -1).UnitVec3()/1.000277
v_right = jtrace.Vec3(0.025, 0, -1).UnitVec3()/1.000277

for v, c in zip([v_left, v_mid, v_right], ['#377eb8', '#ff7f00', '#4daf4a']):
    rays = jtrace.RayVector()
    for p0 in [p0_left, p0_lmid, p0_rmid, p0_right]:
        ray = jtrace.Ray(p0, v, t=0, w=500e-9)
        p0 = ray.positionAtTime(-8)        
        rays.append(jtrace.Ray(p0, v, t=0, w=500e-9))
    trace = telescope.traceFull(rays)
    for surface in trace:
        for inray, outray in zip(surface['inrays'], surface['outrays']):
            plt.plot([inray.x0, outray.x0], [inray.z0, outray.z0], c=c)